In [15]:
import librosa
import mir_eval
import mir_eval.display
import IPython.display
import re
from mido  import MidiFile

In [40]:
from magenta.music.protobuf import music_pb2
import pretty_midi as pm
import magenta.music as mm
import six
#midi = pm.PrettyMIDI("rock.mid")
#mm.midi_to_sequence_proto(midi)

In [17]:
import numpy as np
import pandas as pd

import scipy.stats as stats
import matplotlib.pyplot as plot
import sklearn
import seaborn as sns

#import pretty_midi

import pandas_profiling

#Path for the groove dataset

MIDI_PATH = "/home/mark/repos/Springboard/groove-v1.0.0-midionly/groove/"
DATA_PATH = "/home/mark/repos/Springboard/groove-v1.0.0-midionly/groove/info.csv"

groove_df = pd.read_csv(DATA_PATH)
groove_df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


In [18]:
groove_df['style'].value_counts()

rock                         281
hiphop                        91
funk                          77
punk                          58
neworleans/funk               48
                            ... 
rock/folk                      1
latin/merengue                 1
neworleans/chacha              1
latin/venezuelan-joropo        1
latin/venezuelan-merengue      1
Name: style, Length: 76, dtype: int64

In [19]:
pandas_profiling.ProfileReport(groove_df)

In [20]:
groove_df = groove_df[groove_df.beat_type != 'fill']
short = groove_df[groove_df.duration <=30]
short.count()
#43 < 5 seconds
#158 < 30 seconds
#261 < 1 minute

drummer           158
session           158
id                158
style             158
bpm               158
beat_type         158
time_signature    158
midi_filename     158
audio_filename    148
duration          158
split             158
dtype: int64

In [21]:
groove_df['style'].value_counts()

rock                         193
funk                          36
hiphop                        30
jazz                          24
afrobeat                      13
                            ... 
latin/dominican-merengue       1
latin/venezuelan-merengue      1
afrocuban/rhumba               1
rock/folk                      1
afrocuban/bembe                1
Name: style, Length: 76, dtype: int64

In [22]:
groove_df = groove_df[groove_df.audio_filename.isna() == False]
groove_df.count()

drummer           444
session           444
id                444
style             444
bpm               444
beat_type         444
time_signature    444
midi_filename     444
audio_filename    444
duration          444
split             444
dtype: int64

In [23]:
sample = groove_df.iloc[0]
midi_sample = MIDI_PATH + sample.midi_filename
audio_sample = MIDI_PATH + sample.audio_filename
pm = pm.PrettyMIDI(midi_sample)

In [24]:
groove_df['style'].value_counts()

rock                        171
hiphop                       28
funk                         28
jazz                         24
jazz/swing                   11
                           ... 
neworleans/shuffle            1
latin/dominican-merengue      1
afrocuban/rhumba              1
afrocuban/bembe               1
rock/rockabilly               1
Name: style, Length: 74, dtype: int64

In [25]:
groove_df['style'].value_counts()

rock                        171
hiphop                       28
funk                         28
jazz                         24
jazz/swing                   11
                           ... 
neworleans/shuffle            1
latin/dominican-merengue      1
afrocuban/rhumba              1
afrocuban/bembe               1
rock/rockabilly               1
Name: style, Length: 74, dtype: int64

In [26]:
pandas_profiling.ProfileReport(groove_df)

In [27]:
single_styles = groove_df[groove_df['duration'] <= 20]
single_styles

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
252,drummer1,drummer1/session1,drummer1/session1/76,jazz/mediumfast,180,beat,4-4,drummer1/session1/76_jazz-mediumfast_180_beat_...,drummer1/session1/76_jazz-mediumfast_180_beat_...,7.315270,train
403,drummer1,drummer1/session2,drummer1/session2/33,punk,128,beat,4-4,drummer1/session2/33_punk_128_beat_4-4.mid,drummer1/session2/33_punk_128_beat_4-4.wav,3.811523,train
404,drummer1,drummer1/session2,drummer1/session2/34,punk,128,beat,4-4,drummer1/session2/34_punk_128_beat_4-4.mid,drummer1/session2/34_punk_128_beat_4-4.wav,7.115234,train
406,drummer1,drummer1/session2,drummer1/session2/36,punk,128,beat,4-4,drummer1/session2/36_punk_128_beat_4-4.mid,drummer1/session2/36_punk_128_beat_4-4.wav,7.198242,train
407,drummer1,drummer1/session2,drummer1/session2/37,punk,128,beat,4-4,drummer1/session2/37_punk_128_beat_4-4.mid,drummer1/session2/37_punk_128_beat_4-4.wav,1.821289,train
...,...,...,...,...,...,...,...,...,...,...,...
1127,drummer9,drummer9/session1,drummer9/session1/30,rock,90,beat,4-4,drummer9/session1/30_rock_90_beat_4-4.mid,drummer9/session1/30_rock_90_beat_4-4.wav,18.641676,train
1128,drummer9,drummer9/session1,drummer9/session1/4,rock,100,beat,4-4,drummer9/session1/4_rock_100_beat_4-4.mid,drummer9/session1/4_rock_100_beat_4-4.wav,9.663750,train
1130,drummer9,drummer9/session1,drummer9/session1/6,rock,100,beat,4-4,drummer9/session1/6_rock_100_beat_4-4.mid,drummer9/session1/6_rock_100_beat_4-4.wav,4.748750,train
1132,drummer9,drummer9/session1,drummer9/session1/8,rock,100,beat,4-4,drummer9/session1/8_rock_100_beat_4-4.mid,drummer9/session1/8_rock_100_beat_4-4.wav,11.208750,train


In [28]:
styles = groove_df['style'].value_counts()
style_durations = pd.DataFrame(columns=['style','max','min','sum'])
for s in styles.index:
    style_df = groove_df.query('style=="'+s+'"')
    style_durations = style_durations.append({'style': s, 'max': style_df['duration'].max(), 
                                             'min': style_df['duration'].min(), 'sum': style_df['duration'].sum()},
                                             ignore_index=True)
#print(style_df.count())
style_durations.sort_values(by=['sum'], ascending='True')

,style,max,min,sum
55,latin/merengue,18.304597,18.304597,18.304597
50,jazz/klezmer,37.595410,37.595410,37.595410
58,latin/venezuelan-sangueo,80.779648,80.779648,80.779648
53,latin/reggaeton,87.784634,87.784634,87.784634
67,jazz/fast,105.048858,105.048858,105.048858
...,...,...,...,...
41,latin/brazilian-bossa,611.564048,582.125068,1193.689116
3,jazz,183.415349,3.196875,1383.800351
1,hiphop,191.755389,4.361367,1849.707697
2,funk,297.670000,4.482500,3206.277720


In [29]:
#Method 1: Add the multi-style to the first substyle.
test_df = groove_df
for s in styles.index:
    if '/' in s:
        print(s)
        style_a,style_b = s.split('/')
        print(s.split('/'))
        if style_a in styles.index:
            print(style_a)
            split_style = groove_df.query('style=="'+s+'"')
            test_df = test_df.replace({'style':{s:style_a}})
            #groove_df = groove_df.replace({'style':{s:style_a}})
        if style_b in styles.index:
            print(style_b)
            split_style = groove_df.query('style=="'+s+'"')
            test_df = test_df.replace({'style':{s:style_b}})
            groove_df = groove_df.replace({'style':{s:style_b}})
#groove_df['style'].value_counts()
test_df['style'].value_counts()

jazz/swing
['jazz', 'swing']
jazz
neworleans/funk
['neworleans', 'funk']
funk
latin/brazilian-samba
['latin', 'brazilian-samba']
latin
funk/groove1
['funk', 'groove1']
funk
jazz/fusion
['jazz', 'fusion']
jazz
hiphop/groove6
['hiphop', 'groove6']
hiphop
rock/groove8
['rock', 'groove8']
rock
soul/groove4
['soul', 'groove4']
soul
blues/shuffle
['blues', 'shuffle']
pop/groove7
['pop', 'groove7']
pop
funk/groove5
['funk', 'groove5']
funk
soul/motown
['soul', 'motown']
soul
soul/groove9
['soul', 'groove9']
soul
funk/groove2
['funk', 'groove2']
funk
soul/groove10
['soul', 'groove10']
soul
latin/brazilian
['latin', 'brazilian']
latin
soul/groove3
['soul', 'groove3']
soul
jazz/march
['jazz', 'march']
jazz
rock/prog
['rock', 'prog']
rock
rock/shuffle
['rock', 'shuffle']
rock
dance/disco
['dance', 'disco']
latin/brazilian-baiao
['latin', 'brazilian-baiao']
latin
latin/brazilian-songo
['latin', 'brazilian-songo']
latin
latin/chacarera
['latin', 'chacarera']
latin
latin/brazilian-sambareggae
['lati

rock                     186
funk                      51
jazz                      50
latin                     42
hiphop                    32
soul                      28
pop                       15
punk                       7
afrocuban                  7
reggae                     4
afrobeat                   4
blues/shuffle              4
neworleans/secondline      3
dance/disco                3
dance/breakbeat            2
country                    2
neworleans/shuffle         1
middleeastern              1
neworleans/chacha          1
gospel                     1
Name: style, dtype: int64

In [30]:
#styles = groove_df['style'].value_counts()
styles = test_df['style'].value_counts()
style_durations = pd.DataFrame(columns=['style','max','min','sum'])
for s in styles.index:
    style_df = groove_df.query('style=="'+s+'"')
    style_durations = style_durations.append({'style': s, 'max': style_df['duration'].max(), 
                                             'min': style_df['duration'].min(), 'sum': int(style_df['duration'].sum())},
                                             ignore_index=True)
#print(style_df.count())
style_durations.sort_values(by=['sum'], ascending='True')



,style,max,min,sum
18,neworleans/chacha,115.856863,115.856863,115
16,neworleans/shuffle,130.782434,130.782434,130
19,gospel,139.527083,139.527083,139
14,dance/breakbeat,108.466858,56.392619,164
17,middleeastern,226.604932,226.604932,226
11,blues/shuffle,269.532878,3.564364,288
15,country,154.966071,144.700364,299
6,pop,85.468267,12.709268,313
7,punk,203.251899,1.821289,484
13,dance/disco,225.941667,58.050000,491


In [31]:
styles_removed = style_durations[style_durations['sum'] < 100]
print(styles_removed)
#groove_df = groove_df[groove_df.style]

Empty DataFrame
Columns: [style, max, min, sum]
Index: []


In [44]:
genre_midi_list = {}
for s in styles.index:
    style_df = groove_df.query('style=="'+s+'"')
    genre_midi_list[s] = style_df.midi_filename.tolist()


In [47]:
def midi_to_note_sequence(midi_list):
    note_sequence_list = []
    for file in midi_list:
        midi = pm.PrettyMIDI(MIDI_PATH + file)
        note_sequence = mm.midi_to_sequence_proto(midi)
        note_sequence_list.append(note_sequence)
    return note_sequence_list

In [54]:
#TEST AREA

#frames per sec is something to figure out

#test_midi = pm.PrettyMIDI('/home/mark/repos/Springboard/groove-v1.0.0-midionly/groove/drummer3/session1/4_rock_105_beat_4-4.mid')
#drum_pr = test_midi.get_piano_roll(fs=100)
#print(drum_pr)

#rock_list = midi_to_piano_roll(genre_midi_list['pop'],100)
#print(rock_list)

#midi = pm.PrettyMIDI("rock.mid")
#rock_list = mm.midi_to_sequence_proto(test_midi)

rock_list = midi_to_note_sequence(genre_midi_list['rock'])
#pandas_profiling.ProfileReport(rock_list)
print (type(rock_list[1]))

<class 'music_pb2.NoteSequence'>


In [ ]:
'''
def to_piano_roll(midi):
    """Convert MIDI file to a 2D NumPy ndarray (notes, timesteps)."""
    print (midi)
    notes = 127
    tempo = 500000  # Assume same default tempo and 4/4 for all MIDI files.
    seconds_per_beat = tempo / 1000000.0
    seconds_per_tick = seconds_per_beat / midi.ticks_per_beat
    velocities = np.zeros(notes)
    sequence = []
    for m in midi:
        ticks = int(np.round(m.time / seconds_per_tick))
        ls = [velocities.copy()] * ticks
        sequence.extend(ls)
        if m.type == 'note_on':
            velocities[m.note] = m.velocity
        #elif m.type == 'note_off':
        #    print(m.type)
        #    velocities[m.note] = 0
        else:
            continue
    #print (sequence)
    piano_roll = np.array(sequence).T
    return piano_roll

piano_roll = to_piano_roll(MidiFile('/home/mark/repos/Springboard/groove-v1.0.0-midionly/groove/drummer3/session1/4_rock_105_beat_4-4.mid'))
print (piano_roll)
#import numpy
np.where(piano_roll != 0.)
#lr.display.specshow(piano_roll)
'''